In [28]:
pip install pdfplumber pyspark openai requests bs4 selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 15.3 MB/s eta 0:00:00a 0:00:01
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pdfplumber
import pandas as pd

def extract_pdf_data(file_path):
    """
    Extracts data from the credit card PDF and structures it for a pandas DataFrame.
    """
    data = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            lines = text.split("\n")
            
            # Parse lines for credit card information
            for i in range(len(lines)):
                if "%" in lines[i]:  # Heuristic: look for interest rate
                    # Extract card details (sample parsing logic, adjust as needed)
                    try:
                        card_name = lines[i - 3].strip()
                        bank = lines[i - 2].strip()
                        annual_fee = lines[i + 1].split(" ")[0].strip() if "fee" in lines[i + 1].lower() else "No annual fee"
                        interest_rate = float(lines[i].split("%")[0].strip())
                        rewards = None
                        
                        # Check for rewards (look at subsequent lines)
                        rewards_lines = []
                        j = i + 2
                        while j < len(lines) and ":" in lines[j]:
                            rewards_lines.append(lines[j].split(":")[1].strip())
                            j += 1
                        
                        if rewards_lines:
                            rewards = ", ".join(rewards_lines)
                        
                        # Append to data
                        data.append((card_name, bank, annual_fee, interest_rate, rewards))
                    except Exception as e:
                        print(f"Skipping line due to parsing error: {e}")
                        continue
    return data

# Path to the PDF file
pdf_path = "/Users/aaryas127/Documents/GitHub/credit_card_reward_maximizer/SearchCreditCard-eng.pdf"

# Extract data from the PDF
parsed_data = extract_pdf_data(pdf_path)

# Create a pandas DataFrame from the parsed data
df = pd.DataFrame(parsed_data, columns=["Card Name", "Bank", "Annual Fee", "Purchase Interest Rate (%)", "Rewards"])

# Show the DataFrame
print(df)


                                             Card Name  \
0                                Desjardins Flexi Visa   
1                          MBNA True Line® Mastercard®   
2           Coast Capital Collabria Classic Mastercard   
3                     Tangerine Money-Back Credit Card   
4                           Tangerine World Mastercard   
..                                                 ...   
99          CIBC Aeroplan Visa Infinite Privilege Card   
100           TD Aeroplan Visa Infinite Privilege Card   
101  American Express® Aeroplan®* Reserve Card (Cre...   
102  American Express® Aeroplan®* Reserve Card (Cha...   
103                                  The Platinum Card   

                                                Bank     Annual Fee  \
0                                         Desjardins  No annual fee   
1    MBNA is a division of The Toronto-Dominion Bank  No annual fee   
2                              Coast Capital Savings  No annual fee   
3                  

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Set up ChromeDriver with Service
chrome_driver_path = "/path/to/your/chromedriver"  # Update this with the actual path
service = Service(chrome_driver_path)

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Optional: Run browser in headless mode (without GUI)

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the page
driver.get("https://www.rates.ca/en/credit-cards/td-aeroplan-visa-infinite")

# Allow time for the page to load dynamically
time.sleep(5)

# Get the page source after the JavaScript has executed
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Now you can try to extract the elements again using BeautifulSoup
card_name = soup.find('h1').text if soup.find('h1') else "Not Available"
print("Card Name:", card_name)

# Don't forget to close the driver
driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [32]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://compare.rates.ca/en/credit/cards/343/profile"

# Send a GET request to fetch the page content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Example of extracting a title or heading
    card_name = soup.find('h1').text if soup.find('h1') else "Not Available"
    print("Card Name:", card_name)
else:
    print("Failed to retrieve the page.")

Card Name: Not Available


In [31]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://compare.rates.ca/en/credit/cards/343/profile"

# Send a GET request to fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the relevant information with error handling
def safe_find(tag, class_name=None):
    element = soup.find(tag, class_=class_name) if class_name else soup.find(tag)
    return element.text.strip() if element else "Not Available"

# Extract card details
card_name = safe_find('h1')  # Card name
annual_fee = safe_find('span', 'annual-fee')  # Annual fee (you may need to inspect HTML for this class)
rewards = safe_find('section', 'rewards')  # Rewards info (inspect for correct class)
features = safe_find('section', 'features')  # Features info (inspect for correct class)

# Print the extracted details
print("Card Name:", card_name)
print("Annual Fee:", annual_fee)
print("Rewards:", rewards)
print("Features:", features)


Card Name: Not Available
Annual Fee: Not Available
Rewards: Not Available
Features: Not Available


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

In [ ]:
# Cashback

from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Does the Desjardins Flexi Visa have cashbacks?"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

According to the Desjardins website, the Desjardins Flexi Visa card does offer cashback rewards, but the specifics may vary depending on the card's terms and conditions.

Here's what I found:

* The Desjardins Flexi Visa card offers 1% cashback on most purchases, with no rotating categories or spending limits.
* You can also earn bonus cashback rates on certain categories, such as:
	+ 2% cashback on grocery purchases (some conditions apply)
	+ 2% cashback on recurring bill payments (some conditions apply)
* Cashback rewards are credited to your account annually, in January of each year.

Please note that these details are subject to change, and you should always check the Desjardins website or consult with a representative for the most up-to-date information on the Flexi Visa card's cashback rewards program.

Would you like me to look into any other features or benefits of the Desjardins Flexi Visa card?

In [ ]:
# Travel Benefits

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Does the Desjardins Flexi Visa have any travel benefits?"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

The Desjardins Flexi Visa card does offer some travel-related benefits. Here are a few:

1. **Travel Insurance**: The card comes with travel insurance that covers you for up to 3 days, including trip cancellation, interruption, and delay insurance, as well as travel medical insurance.
2. **Travel Assistance**: You'll have access to a 24/7 travel assistance service, which can help you with things like flight reservations, hotel bookings, and travel documentation.
3. **Purchase Protection and Extended Warranty**: While not exclusively a travel benefit, the card's purchase protection and extended warranty can provide peace of mind when making travel-related purchases, such as luggage or travel gear.
4. **Travel Discounts**: As a Desjardins member, you may be eligible for discounts on travel packages, flights, and hotels through Desjardins' travel agency, Desjardins Travel.

It's worth noting that the Desjardins Flexi Visa card is a basic, no-fee credit card, so the travel benefits are not

In [ ]:
# Signup Bonus, Points, Spending requirement

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Does the Desjardins Flexi Visa have the ability to get you any sort of points?"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

According to the Desjardins website, the Desjardins Flexi Visa card does offer a rewards program, but it's a bit different from traditional points-based programs.

With the Desjardins Flexi Visa, you can earn BONUSDOLLARS, which are redeemable for cash back, travel, or other rewards. Here's how it works:

* You earn 1% BONUSDOLLARS on all purchases made with your card.
* You can redeem your BONUSDOLLARS for:
	+ Cash back: deposited directly into your Desjardins account
	+ Travel: booked through the Desjardins Travel Agency or online
	+ Other rewards: such as gift cards, merchandise, or charitable donations

While the Desjardins Flexi Visa doesn't offer traditional points like some other credit cards, the BONUSDOLLARS program still provides a way to earn rewards on your purchases.

Keep in mind that the rewards program may be subject to change, and there may be some conditions or restrictions on redeeming your BONUSDOLLARS. It's always a good idea to review the terms and conditions of y

In [ ]:
# Insurance

In [ ]:
# Annual fee...first year free?